# Simulating data with PyBullet with ShapeNetSem

Random ShapeNet objects on a ShapeNet table inside a SUNCG room...

Below, we detail the schema for the scene description

`scene_description` schema:
* room_description (for SUNCG house/room)
    * house_id
    * room_id
* table_description
    * mesh_filename
    * position (3D)
    * orientation (quaternion)
    * scale
* object_descriptions (list)
    * mesh_filename (using the ShapeNet subdirectory)
    * position (3D)
    * orientation (quaternion)
    * scale of object

## Imports

In [ ]:
import time
import os, sys
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# my libraries
import simulation_util as sim_util

# pybullet
import pybullet as p
import pybullet_data

# suncg
import pybullet_suncg.simulator as suncg_sim

# for reloading libraries and debugging
from importlib import reload

## Load some SUNCG stuff

In [ ]:
suncg_dir = '/data/suncg/v1/'

# House lists
training_houses_filename = '/data/tabletop_dataset_v4/training_suncg_houses.json'
test_houses_filename = '/data/tabletop_dataset_v4/test_suncg_houses.json'
train_houses = json.load(open(training_houses_filename))
test_houses = json.load(open(test_houses_filename))

# Room types I'm considering
valid_room_types = set(['Living_Room', 'Kitchen', 'Room', 'Dining Room', 'Office'])

# Room objects to filter out
nyuv2_40_classes_filter_list = ['desk', 'chair', 'table', 'person', 'otherstructure', 'otherfurniture']
coarse_grained_classes_filter_list = ['desk', 'chair', 'table', 'person', 'computer', 'bench_chair', 
                                      'ottoman', 'storage_bench', 'pet']

## Loading ShapeNetCore/ShapeNetSem stuff

In [ ]:
shapenet_filepath = '/data/ShapeNetCore.v2/'
shapenetsem_filepath = '/data/ShapeNetSem/models/'

# Create a dictionary of name -> synset_id
temp = json.load(open(shapenet_filepath + 'taxonomy.json'))
taxonomy_dict = {x['name'] : x['synsetId'] for x in temp}
# weirdly, the synsets in the taxonomy file are not the same as what's in the ShapeNetCore.v2 directory. Filter this out
synsets_in_dir = os.listdir(shapenet_filepath)
synsets_in_dir.remove('taxonomy.json')
synsets_in_dir.remove('README.txt')
taxonomy_dict = {k:v for (k,v) in taxonomy_dict.items() if v in synsets_in_dir}

# List of train/test tables
training_tables_filename = '/data/tabletop_dataset_v4/training_shapenet_tables.json'
test_tables_filename = '/data/tabletop_dataset_v4/test_shapenet_tables.json'
train_tables = json.load(open(training_tables_filename))
test_tables = json.load(open(test_tables_filename))

# List of train/test object instances
train_models = pd.read_csv('/data/tabletop_dataset_v4/training_shapenetsem_objects.csv')
test_models = pd.read_csv('/data/tabletop_dataset_v4/test_shapenetsem_objects.csv')

In [ ]:
# debugging. only run if I made a change to SUNCG code, or need to disconnect from PyBullet
sim.disconnect()

In [ ]:
suncg_sim = reload(suncg_sim)

# Generate scenes

In [ ]:
simulation_params = {
    'is_shapenetsem' : True,
    
    # scene stuff
    'min_num_objects_per_scene' : 5,  # 10
    'max_num_objects_per_scene' : 10, # 25
    'simulation_steps' : 500,

    # House stuff
    'house_ids' : train_houses, # test_houses

    # room stuff
    'valid_room_types' : valid_room_types,
    'min_xlength' : 3.0, # Note: I believe this is in meters
    'min_ylength' : 3.0, 

    # table stuff
    'valid_tables' : train_tables, # test_tables
    'max_table_height' : 1.0, # measured in meters
    'min_table_height' : 0.75, 
    'table_init_factor' : 0.9, # this multiplicative factor limits how close you can initialize to wall

    # object stuff
    'object_ids' : train_models, # test_models
    'max_xratio' : 1/3,
    'max_yratio' : 1/3,
    'max_zratio' : 1/4,
    'delta' : 0.2,

    # stuff
    'max_initialization_tries' : 100,

    # Camera/Frustum parameters
    'img_width' : 640, 
    'img_height' : 480,
    'near' : 0.01,
    'far' : 100,
    'fov' : 60, # vertical field of view in angles

    # other camera stuff
    'max_camera_rotation' : np.pi / 10., # Max rotation in radians

    # other stuff
    'taxonomy_dict' : taxonomy_dict,
    'nyuv2_40_classes_filter_list' : nyuv2_40_classes_filter_list,
    'coarse_grained_classes_filter_list' : coarse_grained_classes_filter_list,                   
    
}

In [ ]:
# Create simulator
sim = suncg_sim.Simulator(mode='gui', 
                          suncg_data_dir_base=suncg_dir, 
                          shapenet_data_dir_base=shapenet_filepath, 
                          shapenetsem_data_dir_base=shapenetsem_filepath,
                          params=simulation_params, 
                          verbose=False)

In [ ]:
# Note: This will continue trying to generate a single scene until the task is finished. If it fails,
#       it will spit out why.
def keep_trying_to_generate_single_scene(sim):
    try:
        scenes = sim.generate_scenes(1)
        print("Done")
    except TimeoutError as e:
        print(str(e))
        keep_trying_to_generate_single_scene(sim)
    except Exception as e:
        print("Errored out. Not due to timer, but something else...")
        print(str(e))
        keep_trying_to_generate_single_scene(sim)
                
keep_trying_to_generate_single_scene(sim)

In [ ]:
np.random.seed(1) # debug, to see why textures aren't loading

In [ ]:
# debug
sim.params['simulation_steps'] = 2000

In [ ]:
scenes = sim.generate_scenes(1)

In [ ]:
import pybullet as p
p.connect(p.GUI)

In [ ]:
# vis_mesh_file='/data/ShapeNetSem/models/82cefc73d1ad85e7a2d0d0949f0233b1.obj'
# vis_mesh_file='/data/ShapeNetSem/models/fbb6a1fb01f08eb1575f032b182448e5.obj'
# vis_mesh_file='/data/ShapeNetSem/models/474af011baf759749393793f9cf3dbea.obj'

cid = p.createCollisionShape(p.GEOM_MESH, fileName=vis_mesh_file, meshScale=1)
vid = p.createVisualShape(p.GEOM_MESH, fileName=vis_mesh_file, meshScale=1)

obj_id = p.createMultiBody(baseMass=1, 
                           baseInertialFramePosition=[0,0,0], 
                           baseCollisionShapeIndex=cid, 
                           baseVisualShapeIndex=vid,
                           basePosition=[0,0,0],
                           baseOrientation=[0,0,0,1],
                          )

In [ ]:
p.disconnect()

In [ ]:
obj_id

In [ ]:
p.getAABB(obj_id)

## Actual Data Generation

In [ ]:
def save_img_dict(img_dict, view_num, save_dir):
    # RGB
    rgb_filename = save_dir + f"rgb_{view_num:05d}.jpeg"
    cv2.imwrite(rgb_filename, cv2.cvtColor(img_dict['rgb'], cv2.COLOR_RGB2BGR))

    # Depth
    depth_filename = save_dir + f"depth_{view_num:05d}.png"
    cv2.imwrite(depth_filename, sim_util.saveable_depth_image(img_dict['depth']))

    # Segmentation
    seg_filename = save_dir + f"segmentation_{view_num:05d}.png"
    sim_util.imwrite_indexed(seg_filename, img_dict['seg'].astype(np.uint8))

In [ ]:
total_num_scenarios = 10
views_per_scene = 7
save_path = '/data/tabletop_dataset_v3/training_set/'

scene_num = 0 # start from here
while scene_num < total_num_scenarios:
    
    # Sample scene
    try:
        sim = suncg_sim.Simulator(mode='gui', 
                          suncg_data_dir_base=suncg_dir, 
                          shapenet_data_dir_base=shapenet_filepath, 
                          params=simulation_params, 
                          verbose=False)
        scene_description = sim.generate_scenes(1)[0]
    except TimeoutError as e: # Scene took longer than 45 seconds to generate, or errored out
        print(str(e))
        sim.disconnect()
        continue
    except:
        print("Errored out. Not due to timer, but something else...")
        sim.disconnect()
        continue
        
    # Make directory
    save_dir = save_path + f"scene_{scene_num:05d}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Dictionary to save views
    scene_description['views'] = {}

    # Background-only view
    sim.reset()
    sim.load_house_room(scene_description)
    
    valid_background_view = False
    num_tries = 0
    while not valid_background_view:
        
        if num_tries > simulation_params['max_initialization_tries']:
            break # this will force the entire scene to start over
        num_tries += 1
        
        # Sample the view
        img_dict = sim.sample_room_view()

        # Make sure it's valid. MUST have at least 2 SUNCG objects (e.g. walls/floor/fridge/couch)
        unique_labels = np.unique(img_dict['orig_seg_img'])
        valid_background_view = unique_labels.shape[0] >= 2

    if not valid_background_view:
        print("No valid background views...")
        sim.disconnect()
        continue
    else:
        save_img_dict(img_dict, 0, save_dir)
        scene_description['views']['background'] = img_dict['view_params']
        
    
    # Background-table view
    sim.load_table(scene_description)
    valid_table_view = False
    num_tries = 0
    while not valid_table_view:
        
        if num_tries > simulation_params['max_initialization_tries']:
            break # this will force the entire scene to start over
        num_tries += 1
        
        # Sample the view
        img_dict = sim.sample_table_view()
        
        # Make sure it's valid
        unique_labels = np.unique(img_dict['seg'])
        valid_table_view = 1 in unique_labels
        
    if not valid_table_view:
        print("No valid table views...")
        sim.disconnect()
        continue
    else:
        save_img_dict(img_dict, 1, save_dir)
        scene_description['views']['background+table'] = img_dict['view_params']
        
    # Sample background-table-object views and save 
    sim.load_objects(scene_description)
    scene_description['views']['background+table+objects'] = []
    valid_views = False
    view_num = 2; num_tries = 0    
    while not valid_views: #view_num < views_per_scene:
        
        if num_tries > simulation_params['max_initialization_tries']:
            break # this will force the entire scene to start over
        num_tries += 1

        # Sample the view
        img_dict = sim.sample_table_view()
        
        # Make sure it's valid
        unique_labels = np.unique(img_dict['seg'])
        valid = (0 in unique_labels and # background is in view
                 1 in unique_labels and # table is in view
                 len(set(unique_labels).difference({0,1})) >= 1 # at least 1 objects in view
                )
        if not valid:
            continue # sample another scene

        ### Save stuff ###
        save_img_dict(img_dict, view_num, save_dir)
        scene_description['views']['background+table+objects'].append(img_dict['view_params'])
        
        # increment    
        view_num += 1
        if view_num >= views_per_scene:
            valid_views = True
        
    if not valid_views:
        print("Tried to sample view too many times...")
        sim.disconnect()
        continue    
    
    # Scene Description
    scene_description_filename = save_dir + 'scene_description.txt'
    with open(scene_description_filename, 'w') as save_file:
        json.dump(scene_description, save_file)    

    # increment
    scene_num += 1
    if scene_num % 10 == 0:
        print(f"Generated scene {scene_num}!")
    sim.disconnect()

## Scene saving/loading

In [ ]:
save_filename = '/home/chrisxie/Desktop/scene_description.txt'

In [ ]:
scenes = [sim.export_scene_to_dictionary()]

# Before saving, chop of absolute filepaths
for scene in scenes:
    scene['table']['mesh_filename'] = scene['table']['mesh_filename'].replace(shapenet_filepath, '')
    for object_desc in scene['object_descriptions']:
        object_desc['mesh_filename'] = object_desc['mesh_filename'].replace(shapenet_filepath, '')

# Serialize this JSON file
with open(save_filename, 'w') as save_file:  
    json.dump(scenes, save_file)

In [ ]:
# Load the json file
# scenes = json.load(open(save_filename, 'r'))
scene_description = json.load(open(save_filename, 'r'))[0]

# Load a scene

Given a scene description as I've described, load it in PyBullet

In [ ]:
# scene_description = scenes[0]

In [ ]:
sim.load_scene(scene_description)

## Sample camera positions

In [ ]:
img_dict = sim.sample_table_view()

In [ ]:
# Plot the sampled camera stuff
%matplotlib inline
plt.figure(1, figsize=(20,60))

# RGB image
plt.subplot(1,3,1)
plt.imshow(img_dict['rgb'])
plt.title('RGB')

# Depth image
plt.subplot(1,3,2)
plt.imshow(img_dict['depth'], cmap='gray')
plt.title('Linear depth')

# Segmentation image
print(np.unique(img_dict['seg']))
plt.subplot(1,3,3)
plt.imshow(img_dict['seg'])
plt.title('Segmentation')

# plt.savefig(f'/home/chrisxie/Desktop/ex{i}.png', bbox_inches='tight')
# i += 1

### Debugging stuff

In [ ]:
temp = p.getDebugVisualizerCamera()
camera_yaw, camera_pitch, camera_distance, camera_target = temp[8:]
print("Yaw: {0}".format(camera_yaw))
print("Pitch: {0}".format(camera_pitch))
print("Distance: {0}".format(camera_distance))
print("Target: {0}".format(camera_target))

In [ ]:
# Set camera to look at origin
p.resetDebugVisualizerCamera(1, 0, 0, np.array([0,0,0]))